In [8]:
import pandas as pd
import numpy as np
np.random.seed(123)

In [9]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

In [10]:
df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})

In [11]:
df.to_csv('students.csv')

In [13]:
pd.read_csv('students.csv', index_col=0)

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A
5,John,79,76,93,B
6,Thomas,82,64,81,A
7,Marie,93,63,90,A
8,Albert,92,62,87,A
9,Richard,69,80,94,A


## Indexing and Subsetting

Like the pandas Series object, the pandas DataFrame object supports both position- and label-based indexing using the indexing operator `[]`.

I will demonstrate concrete examples of indexing using the indexing operator `[]` alone and with the `.loc` and `.iloc` attributes below.

### `[]`

I can pass a list of columns from a DataFrame to the indexing operator (aka bracket notation) to return a subset of my original DataFrame.

In [17]:
type(df.name) == type(df['name'])

True

In [18]:
type(df[['name']])

pandas.core.frame.DataFrame

In [26]:
# I can choose a single column using bracket notation.
df['name']

0       Sally
1        Jane
2       Suzie
3       Billy
4         Ada
5        John
6      Thomas
7       Marie
8      Albert
9     Richard
10      Isaac
11       Alan
Name: name, dtype: object

In [27]:
# I can pass a boolean Series to the indexing operator as a selector.
# names that start with a?

In [28]:
bools = df.name.str.lower().str.startswith('a')

In [29]:
df[bools]

,name,math,english,reading,classroom
4,Ada,77,92,98,A
8,Albert,92,62,87,A
11,Alan,92,62,72,A


### `.loc`

We can use the `.loc` attribute to select specific rows AND columns by index label. The index label can be a number, but it can also be a string label. This method offers a lot of flexibility! **The .loc attribute's indexing is inclusive and uses an index label, not position.**

```python
df.loc[row_indexer, column_indexer]
```

In [35]:
my_list = [1,2,3,4,5,6,9]
my_list[:3]

[1, 2, 3]

In [36]:
# Notice the exclusive behavior of the indexing.
df.loc[:4]

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


In [37]:
# Select all the rows and a subset of columns; notice the inclusive behavior of the indexing.



In [38]:
# I can use a boolean Series as a selector with .loc, too, but I can choose rows and columns.
# dimensionality: 0, 1: rows, columns


In [39]:
# [rows, cols]
df.loc[:4, 'math':'reading']

,math,english,reading
0,62,85,80
1,88,79,67
2,94,74,95
3,98,96,88
4,77,92,98


### `.iloc`

We can use the `.iloc` attribute to select specific rows and colums by index position. `.iloc` does not accept a boolean Series as a selector like `.loc` does. **It takes in integers representing index position and is NOT inclusive.**

```python
df.iloc[row_indexer, column_indexer]
```

We can select rows by integer position:

In [40]:
df.iloc[:4]

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B


We can also specify which columns we want to select:

In [44]:
df.iloc[1:4, 2:5]

,english,reading,classroom
1,79,67,B
2,74,95,A
3,96,88,B


Here we select the first 3 rows (everything up to but not including the index of 3), and the second and third columns (starting from the index of 1 up to but not including the index of 3).

## Aggregating

### `.agg`

The `.agg` method lets us specify a way to aggregate a series of numerical values. We pass an aggregate function or list of functions to the method that we want applied to a Series.

In [45]:
df.reading.agg('min')

67

In [47]:
df['reading'].min()

67

In [48]:
df.agg('min')

name         Ada
math          62
english       62
reading       67
classroom      A
dtype: object

In [49]:
# I can pass a list of functions to the .agg method.
df[['math', 'english']].agg(['max', 'min'])


,math,english
max,98,99
min,62,62


While on the surface this seems pretty simple, `.agg` is capable of providing more detailed aggregations:

In [50]:
df.describe()

,math,english,reading
count,12.000000,12.000000,12.000000
mean,84.833333,77.666667,86.500000
std,11.134168,13.371158,9.643651
min,62.000000,62.000000,67.000000
25%,78.500000,63.750000,80.750000
50%,90.000000,77.500000,89.000000
75%,92.250000,86.750000,93.250000
max,98.000000,99.000000,98.000000


In [52]:
df[['math', 'english']].agg(['max', 'min', 'mean', 'std']).T

,max,min,mean,std
math,98.0,62.0,84.833333,11.134168
english,99.0,62.0,77.666667,13.371158


### `.groupby`

The `.groupby` method is used to create a grouped object, which we can then apply an aggregation on. For example, if we wanted to know the highest math grade from each classroom:

In [57]:
# get our maximum grade for our 'math' column for classroom A and B respectively

In [56]:
df.groupby('classroom').max()

,name,math,english,reading
classroom,,,,
A,Thomas,94,92,98
B,John,98,99,93


We can use `.agg` here to, to see multiple aggregations:

In [58]:
# structure:
# take your dataframe df
# group by classroom column
# use the math column --> referenced with .math (could use ['math'] instead)
# use a math method --> aggregation via .agg
# use these functions in agg: min, mean, max


In [59]:
df.groupby('classroom').agg(['min', 'mean', 'max'])

/var/folders/tg/htdzmdmj40vgs24scqz284xm0000gn/T/ipykernel_54936/1999846839.py:1: FutureWarning: ['name'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.groupby('classroom').agg(['min', 'mean', 'max'])


math             english            reading            
           min    mean max     min   mean max     min    mean max
classroom                                                        
A           62  82.625  94      62  72.75  92      72  87.125  98
B           79  89.250  98      76  87.50  99      67  85.250  93

We can group by multiple columns as well. To demonstrate, we'll create a boolean column named `passing_math`, then group by the combination of our new feature, `passing_math`, and the classroom and calculate the average reading grade and the number of individuals in each subgroup. 

Let's break this problem down and code it step-by-step.

#### `np.where`

First, we can create the new `passing_math` column using a handy NumPy function called `np.where`. It will allow us to base the new column values on whether the values in an existing column, `math`, meet a condition.

```python
np.where(condition, this_where_True, this_where_False)
```

In [60]:
df.math >= 70

0     False
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9     False
10     True
11     True
Name: math, dtype: bool

In [62]:
df

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A
5,John,79,76,93,B
6,Thomas,82,64,81,A
7,Marie,93,63,90,A
8,Albert,92,62,87,A
9,Richard,69,80,94,A


In [63]:
df['passing_math'] = np.where(df.math >= 70, "Pass", "Fail")

In [65]:
df.head()

,name,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,Fail
1,Jane,88,79,67,B,Pass
2,Suzie,94,74,95,A,Pass
3,Billy,98,96,88,B,Pass
4,Ada,77,92,98,A,Pass


In [66]:
#same as
#df['fake_passing_math'] = df['math'].apply(lambda x: 'Pass' if x>= 70 else 'Fail')

Argument descriptors:
np.where(condition, thing if true, thing if false)
condition: math grade is less than 70 --> df.math < 70
output if True: 'failing'
output if False: 'passing'

Next, we will group by the `passing_math` and `classroom` columns and use the `.agg` method to calculate the average reading grade and the number of students.

In [69]:
grade_groups = df.groupby(['classroom', 'passing_math']).reading.agg(['mean', 'count'])

In [70]:
grade_groups

mean  count
classroom passing_math                  
A         Fail          87.000000      2
          Pass          87.166667      6
B         Pass          85.250000      4

**Takeaways:**

We can interpret this output as there being 2 students failing math in classroom A with an average reading grade of 87, 6 students passing math in classroom A with an average reading grade of 87.16, and 4 students passing math in classroom B with an average reading grade of 85.25.

#### `.transform`

The `.transform` method can be used to produce a series with the same length of the original dataframe where each value represents the aggregation from the subgroup resulting from the `.groupby`. 

For example, if we wanted to know the average math grade for each classroom and add this data back to our original dataframe:

In [71]:
# get avg math grade for each classroom
#df.groupby('classroom').math.mean()
df.assign(avg_math_by_class=df.groupby('classroom').math.transform('mean'))

,name,math,english,reading,classroom,passing_math,avg_math_by_class
0,Sally,62,85,80,A,Fail,82.625
1,Jane,88,79,67,B,Pass,89.250
2,Suzie,94,74,95,A,Pass,82.625
3,Billy,98,96,88,B,Pass,89.250
4,Ada,77,92,98,A,Pass,82.625
5,John,79,76,93,B,Pass,89.250
6,Thomas,82,64,81,A,Pass,82.625
7,Marie,93,63,90,A,Pass,82.625
8,Albert,92,62,87,A,Pass,82.625
9,Richard,69,80,94,A,Fail,82.625


#### `.describe`

Check out what I can do when I combine a `.groupby` with a `.describe`!

In [74]:
df.groupby(['classroom', 'passing_math']).describe().T

classroom              A                     B
passing_math        Fail       Pass       Pass
math    count   2.000000   6.000000   4.000000
        mean   65.500000  88.333333  89.250000
        std     4.949747   7.061633   7.973916
        min    62.000000  77.000000  79.000000
        25%    63.750000  84.500000  85.750000
        50%    65.500000  92.000000  90.000000
        75%    67.250000  92.750000  93.500000
        max    69.000000  94.000000  98.000000
english count   2.000000   6.000000   4.000000
        mean   82.500000  69.500000  87.500000
        std     3.535534  11.928956  11.676187
        min    80.000000  62.000000  76.000000
        25%    81.250000  62.250000  78.250000
        50%    82.500000  63.500000  87.500000
        75%    83.750000  71.500000  96.750000
        max    85.000000  92.000000  99.000000
reading count   2.000000   6.000000   4.000000
        mean   87.000000  87.166667  85.250000
        std     9.899495   9.537645  12.392874
        min    80.000000  72.000000  67.000000
        25%    83.500000  82.500000  82.750000
        50%    87.000000  88.500000  90.500000
        75%    90.500000  93.750000  93.000000
        max    94.000000  98.000000  93.000000

## Merging and Joining

Pandas provides several ways to combine dataframes together. We will look at two of them below:

### `pd.concat`

This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis, row-wise axis=0 or column-wise axis=1.

```python
# For example, concat with a list of two DataFrames
pd.concat([df1, df2], axis=0)
```

- When your list contains at least one DataFrame, a DataFrame is returned.


- When concatenating only Series objects row-wise, axis=0, a Series is returned.


- When concatenating Series or DataFrames column-wise, axis=1, a DataFrame is returned.

```python
# Default is set to row-wise concatenation using an outer join.
pd.concat(objs, axis=0, join='outer')
```

When concatenating dataframes vertically, we basically are just adding more rows to an existing dataframe. In this case, the dataframes we are putting together should have the same column names[^1].

In [76]:
c1 = pd.DataFrame({'col1':[1,2,3], 'col2' : [4,5,6]})
c2 = pd.DataFrame({'col1':[14,26,33], 'col2' : [44,65,86]})

In [77]:
c1

,col1,col2
0,1,4
1,2,5
2,3,6


In [78]:
c2

,col1,col2
0,14,44
1,26,65
2,33,86


In [82]:
pd.concat([c1, c2], axis=1)

,col1,col2,col1,col2
0,1,4,14,44
1,2,5,26,65
2,3,6,33,86


**Note** that the indices are preserved on the resulting dataframe; we could set the `ignore_index` parameter to `True` if we wanted these to be sequential.

In [83]:
pd.concat([c1, c2], axis=1, ignore_index=True)

,0,1,2,3
0,1,4,14,44
1,2,5,26,65
2,3,6,33,86


[^1]:
    We can concatenate dataframes with different column names, but generally this is not the behavior we want, as pandas will fill in a lot of null values into the resulting dataframe. The exception to this is if the dataframes are aligned on their index (i.e. the labels for each row), then we can provide the `axis=1` keyword argument to `pd.concat` to merge the dataframes horizontally.

### `.merge`

This method is similar to a SQL join. Here's a [cool read](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join) making a comparison between the two, if you're interested.

```python
# df.merge default settings for commonly used parameters.

left_df.merge(right_df, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)
```

How does changing the default argument of the `how` parameter change my resulting DataFrame?

`how` == Type of merge to be performed.

`how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.

`how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.

`how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

`how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [84]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [85]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


The `.merge` method will allow us to specify `left_on` and `right_on` to indicate the columns that are the keys used to merge the dataframes together. 

- In addition, the `how` keyword argument is used to define what type of JOIN we want to do; as we saw above, `inner` is the default setting. 

- For demonstration purposes, I'm also going to set the `indicator` parameter to `True`, which will create a column indicating whether the merge key appears in the `left_only`, `right_only` or `both` DataFrames.

In [86]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


Notice that we have duplicate column names in the resulting dataframe. By default, pandas will add a suffix of `_x` to any columns in the left dataframe that are duplicated, and `_y` to any columns in the right dataframe that are duplicated. I can clean up my columns if I want to; one way would be to use method chaining, which it demonstrated below:

In [87]:
# left_on = left table key
# right_on = right table key

# ON left.key = right.key


In [89]:
temp = (users.merge(roles, 
            left_on='role_id', 
            right_on='id')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)
temp

,id,employee,role_id,role
0,1,bob,1,admin
1,2,joe,2,author
2,3,sally,3,reviewer
3,4,adam,3,reviewer


In [90]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4 non-null      int64 
 1   employee  4 non-null      object
 2   role_id   4 non-null      int64 
 3   role      4 non-null      object
dtypes: int64(2), object(2)
memory usage: 160.0+ bytes


## Exercises II

1. Copy the `users` and `roles` DataFrames from the examples above. 

2. What is the result of using a `right` join on the DataFrames? 

3. What is the result of using an `outer` join on the DataFrames?
     
4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

5. Load the `mpg` dataset from PyDataset. 

6. Output and read the documentation for the `mpg` dataset.

7. How many rows and columns are in the dataset?

8. Check out your column names and perform any cleanup you may want on them.

9. Display the summary statistics for the dataset.

10. How many different manufacturers are there?

11. How many different models are there?

12. Create a column named `mileage_difference` like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

13. Create a column named `average_mileage` like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

14. Create a new column on the `mpg` dataset named `is_automatic` that holds boolean values denoting whether the car has an automatic transmission.

15. Using the `mpg` dataset, find out which which manufacturer has the best miles per gallon on average?

16. Do automatic or manual cars have better miles per gallon?